# Load Models and Generate the submission File

### This notebook will show how to
- load the checkpoints of our baseline models
- use our API to generate a submission .csv file
- use the .csv file to make a submission

We also show how
- we extract the ground truth responses from a dataset
- we use these ground truth responses and the submitted files to calculate the score of the live leaderboard of our competition

In [4]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from nnfabrik.builder import get_data, get_model, get_trainer

In [5]:
tier = 'train'
dataset_name = '20457-5-9-0'

filenames = ['./data/lurz2020/static20457-5-9-preproc0', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': True,
                 'batch_size': 128,
                 'exclude': None,
                 'file_tree': True,
                 'scale': 1,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

dataport not available, will only be able to load data locally


In [6]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 4,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_dilation': 1,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
               }

sota_model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

sota_model.load_state_dict(torch.load("./checkpoints/sota_model.pth"));

/src/neuralpredictors/neuralpredictors/layers/readouts/base.py:72: UserWarning: Use of 'gamma_readout' is deprecated. Please consider using the readout's feature-regularization parameter instead
  warnings.warn(
/src/neuralpredictors/neuralpredictors/layers/readouts/base.py:88: UserWarning: Readout is NOT initialized with mean activity but with 0!
  warnings.warn("Readout is NOT initialized with mean activity but with 0!")


In [7]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 3,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_dilation': 1,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
              'linear': True
               }
ln_model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

ln_model.load_state_dict(torch.load("./checkpoints/ln_model.pth"));

# Generate the Submission File with our API

In [8]:
# import the API from the competition repo
from sensorium.utility import submission

In [10]:
# generate the submission file
submission.generate_submission_file(trained_model=sota_model, 
                                    test_dataloader=dataloaders["test"][dataset_name], 
                                    path='./submission_files/sota_submission_file.csv')

File saved.


In [13]:
# identical procedure with the LN model
submission.generate_submission_file(trained_model=ln_model, 
                                    test_dataloader=dataloaders["test"][dataset_name], 
                                    path='./submission_files/ln_submission_file.csv')

File saved.


### Inspect content of submission file

In [11]:
import pandas as pd
pd.read_csv('./submission_files/sota_submission_file.csv')

,trial_indices,image_ids,prediction,neuron_ids
0,126,2214,"[0.4129704236984253, 0.8013747930526733, 0.392...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
1,297,2214,"[0.43444764614105225, 0.9675015211105347, 0.39...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
2,597,2214,"[0.4244424104690552, 0.8653027415275574, 0.450...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
3,852,2214,"[0.44398176670074463, 0.9990906715393066, 0.41...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
4,908,2214,"[0.42371731996536255, 0.9457056522369385, 0.46...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
...,...,...,...,...
994,2752,3487,"[0.12793129682540894, 0.20165026187896729, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
995,3039,3487,"[0.12709170579910278, 0.19649678468704224, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
996,4312,3487,"[0.13403362035751343, 0.23714011907577515, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
997,4380,3487,"[0.13138562440872192, 0.20400822162628174, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."


In [14]:
import pandas as pd
pd.read_csv('./submission_files/ln_submission_file.csv')

,trial_indices,image_ids,prediction,neuron_ids
0,126,2214,"[0.26144659519195557, 0.5331838726997375, 0.49...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
1,297,2214,"[0.2594672441482544, 0.5381501913070679, 0.502...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
2,597,2214,"[0.2711303234100342, 0.5490167737007141, 0.487...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
3,852,2214,"[0.24851012229919434, 0.5322059988975525, 0.49...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
4,908,2214,"[0.26896047592163086, 0.49697959423065186, 0.4...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
...,...,...,...,...
994,2752,3487,"[0.2704678177833557, 0.7939815521240234, 1.037...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
995,3039,3487,"[0.2738579511642456, 0.7711479663848877, 1.059...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
996,4312,3487,"[0.27059680223464966, 0.8960937261581421, 1.04...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
997,4380,3487,"[0.2709449529647827, 0.8625759482383728, 0.958...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."


# The submission process

- now that the .csv file is generated, the participant can make an entry to the competition
- all that is left to do is:
 - register a new user on http://sensorium2022.net/ with a valid email address
 - upload the .csv file
 - the competition score will be calculated automatically and will show up in the live leaderboard within a few seconds

---

# How the scores get computed 

- we will withhold the ground truth neuronal responses to the test set images in the actual competition
- here we show 
 - how we extract the ground truth responses from the demo dataset (where the test set responses are present)
 - how the metrics of the competition are calculated

In [16]:
from sensorium.eval import evaluate

In [18]:
filename = ['./data/lurz2020/static20457-5-9-preproc0']
submission.generate_ground_truth_file(filename=filename,
                                      path='./ground_truth_files/ground_truth_file.csv')

### SOTA model results

In [19]:
ground_truth_file = './ground_truth_files/ground_truth_file.csv'
submission_file = './submission_files/sota_submission_file.csv'

In [20]:
out = evaluate(submission_file, ground_truth_file)

In [21]:
print("Results for the SOTA model:")
for metric, value in out.items():
    print(f"{metric}: {np.round(value, 3)}")

Results for the SOTA model:
Correlation (single trial): 0.296
Correlation (mean): 0.539
FEVE: 0.292


### LN model results

In [22]:
ground_truth_file = './ground_truth_files/ground_truth_file.csv'
submission_file = './submission_files/ln_submission_file.csv'

In [23]:
out = evaluate(submission_file, ground_truth_file)

In [24]:
print("Results for the LN model:")
for metric, value in out.items():
    print(f"{metric}: {np.round(value,3)}")

Results for the LN model:
Correlation (single trial): 0.203
Correlation (mean): 0.365
FEVE: 0.129


---